In [3]:
import numpy as np
import os
import tensorflow as tf
from data_utils import minibatches, pad_sequences, get_chunks
from general_utils import Progbar, print_sentence

from data_utils import get_trimmed_glove_vectors, load_vocab, \
    get_processing_word, CoNLLDataset
from model import NERModel
from config import Config

# Sequence Tagging (Named Entity Recognition) with Tensorflow:
Implement a sequence tagging model using tensorflow. Apparently this demo has state-of-the-art performance (F1 score close to 91) given a large enough corpus.

## Dependencies
https://github.com/stanfordnlp/GloVe
## Task

Given a sentence, give a tag to each word. A classical application is Named Entity Recognition (NER). 

According to Wikipedia:
Named-entity recognition (NER) (also known as entity identification, entity chunking and entity extraction) is a subtask of information extraction that seeks to locate and classify named entities in text into pre-defined categories such as the names of persons, organizations, locations, expressions of times, quantities, monetary values, percentages, etc.

Here is an example:

```
John  lives in New   York  and works for the European Union
B-PER O     O  B-LOC I-LOC O   O     O   O   B-ORG    I-ORG
```

## Entity Notation

In the CoNLL2003 task, the entities are LOC, PER, ORG and MISC for locations, persons, orgnizations and miscellaneous. The no-entity tag is O. Because some entities (like New York) have multiple words, we use a tagging scheme to distinguish between the beginning (tag B-...), or the inside of an entity (tag I-...). Other tagging schemes exist (IOBES, etc). Tagging schemes are arbitrarily determined.


## Very High Level Overview of Model

Word Representation: Word vector w \in R^{n}. In this case, we're using GloVe, but other word embeddings include Word2Vec, Senna, etc.

Contextual Word Representation: Use an LSTM to represent for each word in its context (via hidden layers).

Decoding: Once we have a vector representing each word, we can use it to make a prediction.

## Lower Level Overview of Model
Similar to [Lample et al.](https://arxiv.org/abs/1603.01360) and [Ma and Hovy](https://arxiv.org/pdf/1603.01354.pdf).

- concatenate final states of a bi-lstm on character embeddings to get a character-based representation of each word
- concatenate this representation to a standard word vector representation (GloVe here)
- run a bi-lstm on each sentence to extract contextual representation of each word
- decode with a linear chain CRF

## Steps
Follow Step 1., ..., Step n. to complete the model.

blog post as reference: https://guillaumegenthial.github.io/sequence-tagging-with-tensorflow.html

In [2]:
class NERModel(object):
    def __init__(self, config, embeddings, ntags, nchars=None):
        """
        Args:
            config: class with hyper parameters
            embeddings: np array with embeddings
            nchars: (int) size of chars vocabulary
        """
        self.config     = config
        self.embeddings = embeddings
        self.nchars     = nchars
        self.ntags      = ntags
        self.logger     = config.logger # now instantiated in config

    def add_placeholders(self):
        """
        Adds placeholders to self
        """

        """
        Step 1.a. Since Tensorflow recieves batches of words and data, pad sentences to
        make them the same length. define two placeholders (self.word_ids, self.sequence_lengths).
        We pad according to the max length of a word in data/ and max length of a sentence in data/.
        This is already been done for the sake of syntax.
        """
        # shape = (batch size, max length of sentence in batch)
        self.word_ids = tf.placeholder(tf.int32, shape=[None, None],
                        name="word_ids")

        # shape = (batch size)
        self.sequence_lengths = tf.placeholder(tf.int32, shape=[None],
                        name="sequence_lengths")

        # shape = (batch size, max length of sentence, max length of word)
        self.char_ids = tf.placeholder(tf.int32, shape=[None, None, None],
                        name="char_ids")

        # shape = (batch_size, max_length of sentence)
        self.word_lengths = tf.placeholder(tf.int32, shape=[None, None],
                        name="word_lengths")

        # shape = (batch size, max length of sentence in batch)
        self.labels = tf.placeholder(tf.int32, shape=[None, None],
                        name="labels")

        # hyper parameters
        self.dropout = tf.placeholder(dtype=tf.float32, shape=[],
                        name="dropout")
        self.lr = tf.placeholder(dtype=tf.float32, shape=[], 
                        name="lr")


    def get_feed_dict(self, words, labels=None, lr=None, dropout=None):
        """
        Given some data, pad it and build a feed dictionary
        Args:
            words: list of sentences. A sentence is a list of ids of a list of words. 
                A word is a list of ids
            labels: list of ids
            lr: (float) learning rate
            dropout: (float) keep prob
        Returns:
            dict {placeholder: value}
        """
        # perform padding of the given data
        if self.config.chars:
            char_ids, word_ids = zip(*words)
            word_ids, sequence_lengths = pad_sequences(word_ids, 0)
            char_ids, word_lengths = pad_sequences(char_ids, pad_tok=0, nlevels=2)
        else:
            word_ids, sequence_lengths = pad_sequences(words, 0)

        # build feed dictionary
        feed = {
            self.word_ids: word_ids,
            self.sequence_lengths: sequence_lengths
        }

        if self.config.chars:
            feed[self.char_ids] = char_ids
            feed[self.word_lengths] = word_lengths

        if labels is not None:
            labels, _ = pad_sequences(labels, 0)
            feed[self.labels] = labels

        if lr is not None:
            feed[self.lr] = lr

        if dropout is not None:
            feed[self.dropout] = dropout

        return feed, sequence_lengths

    """
    Step 1.b. Use a bi-LSTM for feature extraction of words at the n-gram level
    (so output a word embedding for each word).
    """
    def add_word_embeddings_op(self):
        """
        Adds word embeddings to self
        """
        with tf.variable_scope("words"):
            _word_embeddings = tf.Variable(self.embeddings, name="_word_embeddings", dtype=tf.float32, 
                                trainable=self.config.train_embeddings)
            word_embeddings = tf.nn.embedding_lookup(_word_embeddings, self.word_ids, 
                name="word_embeddings")

        with tf.variable_scope("chars"):
            if self.config.chars:
                """
                Initialize an embedding matrix that is like a dictionary of
                character: letter vector. 
                Set the shape of the output _char_embeddings accordingly to:
                self.nchars- The number of characters available in vocab.
                    This is a predefined instance variable.
                self.config.dim_char- The dimension of a character vector.
                    This is a predefined instance variable.
                    
                char_embeddings is the actual matrix to look up the letter vectors.
                """
                _char_embeddings = tf.get_variable(name="_char_embeddings", dtype=tf.float32, 
                    shape=['-----', '-----'])
                # shape = (batch, sentence, word, dim of char embeddings)
                char_embeddings = tf.nn.embedding_lookup(_char_embeddings, self.char_ids, 
                    name="char_embeddings")
                
                """
                Reshape our 4-dimensional tensor to match the requirement of bidirectional_dynamic_rnn.
                This has been done for you because messing with shapes is sad.
                """
                # put the time dimension on axis=1 for dynamic_rnn
                s = tf.shape(char_embeddings) # store old shape
                # shape = (batch x sentence, word, dim of char embeddings)
                char_embeddings = tf.reshape(char_embeddings, shape=[-1, s[-2], self.config.dim_char])
                word_lengths = tf.reshape(self.word_lengths, shape=[-1])
                """
                Initialize a bi lstm on characters. Make
                cell_fw: An instance of tf.contrib.rnn.LSTMCell, to be used for forward direction.
                cell_bw: An instance of tf.contrib.rnn.LSTMCell, to be used for backward direction.
                
                where hidden state size is self.config.char_hidden_size.
                The state of the lstm should a tuple of memory and hidden state.
                """
                # need 2 instances of cells since tf 1.1
                cell_fw = '-----'
                cell_bw = '-----'

                _, ((_, output_fw), (_, output_bw)) = tf.nn.bidirectional_dynamic_rnn(cell_fw, 
                    cell_bw, char_embeddings, sequence_length=word_lengths, 
                    dtype=tf.float32)

                output = tf.concat([output_fw, output_bw], axis=-1)
                # shape = (batch size, max sentence length, char hidden size)
                output = tf.reshape(output, shape=[-1, s[1], 2*self.config.char_hidden_size])

                word_embeddings = tf.concat([word_embeddings, output], axis=-1)

        self.word_embeddings =  tf.nn.dropout(word_embeddings, self.dropout)

    """
    Step 2.a. Decoding
    Computing Tags Scores: At this stage, each word w is associated with a vector h that 
    captures information from the meaning of the word, its characters and its context.
    """
    def add_logits_op(self):
        """
        Adds logits to self
        """
        """
        Here is a bi-lstm for your bi-lstm just in case you didn't have enough.
        Predicts the output vector for each word embedding. Like characters,
        words are read sequentially so it'd also be a good idea to 
        analyze them with a bi-lstm. 
        
        Create
        cell_fw, cell_bw: Instances of tf.contrib.rnn.LSTMCell.
            To be used for forward direction.
            Hidden state size = self.config.hidden_size
        (output_fw, output_bw), _: outputs of tf.nn.bidirectional_dynamic_rnn.
            Like the bi-lstm for char embeddings, the bi-lstm of word embeddings
            should include cell_fw, cell_bw, input, sequence length (sentence length).
        """
        with tf.variable_scope("bi-lstm"):
            cell_fw = '-----'
            cell_bw = '-----'
            (output_fw, output_bw), _ = tf.nn.bidirectional_dynamic_rnn('-----', 
                '-----', '-----', sequence_length='-----', 
                dtype=tf.float32)
            # Stores the output as a tuple of the forward and backward direction
            # output states
            output = tf.concat([output_fw, output_bw], axis=-1)
            """
            Implement dropout with tf.nn.dropout with hyperparameter, self.dropout.
            """
            output = '-----'
        """
        We compute tags scores as self.logits, where each element in vector 
        This is to make a final prediction. Use a fully connected neural network
        (a vanilla 189-esque neural net will suffice) to get a vector where each entry
        corresponds to a score for a specific tag.
        We interpret the ith element as the score of class i for word w.
        This has already been done.
        """
        with tf.variable_scope("proj"):
            W = tf.get_variable("W", shape=[2*self.config.hidden_size, self.ntags], 
                dtype=tf.float32)

            b = tf.get_variable("b", shape=[self.ntags], dtype=tf.float32, 
                initializer=tf.zeros_initializer())

            ntime_steps = tf.shape(output)[1]
            output = tf.reshape(output, [-1, 2*self.config.hidden_size])
            pred = tf.matmul(output, W) + b
            self.logits = tf.reshape(pred, [-1, ntime_steps, self.ntags])


    def add_pred_op(self):
        """
        Adds labels_pred to self
        """
        if not self.config.crf:
            self.labels_pred = tf.cast(tf.argmax(self.logits, axis=-1), tf.int32)

    """
    Step 2.b. We want to compute the probability of a tagging sequence y_t
    and find the sequence with the highest probability. here, y_t is the id
    of the tag for the t-th word.
    
    We have two options to make our final prediction:
    Softmax: logistic function that squashes scores to range [0,1] that add up to 1.
        Calculate losses with tf.nn.sparse_softmax_cross_entropy_with_logits.
        Calculate self.loss with tf.reduce_mean to normalize the negative log likelihood
            as a probability distribution.
        Available parameters: self.logits, self.labels, self.sequence_lengths.
    Linear-chain CRF: makes use of the neighboring tag decisions. Finds a sequence of tags
    with the best score by scaling scores of intermediate tags with a transition matrix/
    probability distribution.
        Calculate log_likelihood, self.transition_params with tf.contrib.crf.crf_log_likelihood.
        Calculate mask with tf.sequence_mask.
        Calculate losses with tf.boolean_mask from previous loss and mask.
        Calculate self.loss with tf.reduce_mean to normalize the losses 
            as a probability distribution.
        Available parameters: self.logits, self.labels, self.sequence_lengths.
        
    """
    def add_loss_op(self):
        """
        Adds loss to self
        """
        if self.config.crf:
            log_likelihood, self.transition_params = '-----'
            self.loss = '-----'
        else:
            losses = '-----'
            mask = '-----'
            losses = '-----'
            self.loss = '-----'

        # for tensorboard
        tf.summary.scalar("loss", self.loss)


    def add_train_op(self):
        """
        Add train_op to self
        """
        with tf.variable_scope("train_step"):
            # sgd method
            if self.config.lr_method == 'adam':
                optimizer = tf.train.AdamOptimizer(self.lr)
            elif self.config.lr_method == 'adagrad':
                optimizer = tf.train.AdagradOptimizer(self.lr)
            elif self.config.lr_method == 'sgd':
                optimizer = tf.train.GradientDescentOptimizer(self.lr)
            elif self.config.lr_method == 'rmsprop':
                optimizer = tf.train.RMSPropOptimizer(self.lr)
            else:
                raise NotImplementedError("Unknown train op {}".format(
                                          self.config.lr_method))

            # gradient clipping if config.clip is positive
            if self.config.clip > 0:
                gradients, variables   = zip(*optimizer.compute_gradients(self.loss))
                gradients, global_norm = tf.clip_by_global_norm(gradients, self.config.clip)
                self.train_op = optimizer.apply_gradients(zip(gradients, variables))
            else:
                self.train_op = optimizer.minimize(self.loss)


    def add_init_op(self):
        self.init = tf.global_variables_initializer()


    def add_summary(self, sess): 
        # tensorboard stuff
        self.merged = tf.summary.merge_all()
        self.file_writer = tf.summary.FileWriter(self.config.output_path, sess.graph)


    def build(self):
        self.add_placeholders()
        self.add_word_embeddings_op()
        self.add_logits_op()
        self.add_pred_op()
        self.add_loss_op()
        self.add_train_op()
        self.add_init_op()


    def predict_batch(self, sess, words):
        """
        Args:
            sess: a tensorflow session
            words: list of sentences
        Returns:
            labels_pred: list of labels for each sentence
            sequence_length
        """
        # get the feed dictionnary
        fd, sequence_lengths = self.get_feed_dict(words, dropout=1.0)

        if self.config.crf:
            viterbi_sequences = []
            logits, transition_params = sess.run([self.logits, self.transition_params], 
                    feed_dict=fd)
            # iterate over the sentences
            for logit, sequence_length in zip(logits, sequence_lengths):
                # keep only the valid time steps
                logit = logit[:sequence_length]
                viterbi_sequence, viterbi_score = tf.contrib.crf.viterbi_decode(
                                logit, transition_params)
                viterbi_sequences += [viterbi_sequence]

            return viterbi_sequences, sequence_lengths

        else:
            labels_pred = sess.run(self.labels_pred, feed_dict=fd)

            return labels_pred, sequence_lengths


    def run_epoch(self, sess, train, dev, tags, epoch):
        """
        Performs one complete pass over the train set and evaluate on dev
        Args:
            sess: tensorflow session
            train: dataset that yields tuple of sentences, tags
            dev: dataset
            tags: {tag: index} dictionary
            epoch: (int) number of the epoch
        """
        nbatches = (len(train) + self.config.batch_size - 1) // self.config.batch_size
        prog = Progbar(target=nbatches)
        for i, (words, labels) in enumerate(minibatches(train, self.config.batch_size)):
            fd, _ = self.get_feed_dict(words, labels, self.config.lr, self.config.dropout)

            _, train_loss, summary = sess.run([self.train_op, self.loss, self.merged], feed_dict=fd)

            prog.update(i + 1, [("train loss", train_loss)])

            # tensorboard
            if i % 10 == 0:
                self.file_writer.add_summary(summary, epoch*nbatches + i)

        acc, f1 = self.run_evaluate(sess, dev, tags)
        self.logger.info("- dev acc {:04.2f} - f1 {:04.2f}".format(100*acc, 100*f1))
        return acc, f1


    def run_evaluate(self, sess, test, tags):
        """
        Evaluates performance on test set
        Args:
            sess: tensorflow session
            test: dataset that yields tuple of sentences, tags
            tags: {tag: index} dictionary
        Returns:
            accuracy
            f1 score
        """
        accs = []
        correct_preds, total_correct, total_preds = 0., 0., 0.
        for words, labels in minibatches(test, self.config.batch_size):
            labels_pred, sequence_lengths = self.predict_batch(sess, words)

            for lab, lab_pred, length in zip(labels, labels_pred, sequence_lengths):
                lab = lab[:length]
                lab_pred = lab_pred[:length]
                accs += [a==b for (a, b) in zip(lab, lab_pred)]
                lab_chunks = set(get_chunks(lab, tags))
                lab_pred_chunks = set(get_chunks(lab_pred, tags))
                correct_preds += len(lab_chunks & lab_pred_chunks)
                total_preds += len(lab_pred_chunks)
                total_correct += len(lab_chunks)

        p = correct_preds / total_preds if correct_preds > 0 else 0
        r = correct_preds / total_correct if correct_preds > 0 else 0
        f1 = 2 * p * r / (p + r) if correct_preds > 0 else 0
        acc = np.mean(accs)
        return acc, f1


    def train(self, train, dev, tags):
        """
        Performs training with early stopping and lr exponential decay

        Args:
            train: dataset that yields tuple of sentences, tags
            dev: dataset
            tags: {tag: index} dictionary
        """
        best_score = 0
        saver = tf.train.Saver()
        # for early stopping
        nepoch_no_imprv = 0
        with tf.Session() as sess:
            sess.run(self.init)
            if self.config.reload:
                self.logger.info("Reloading the latest trained model...")
                saver.restore(sess, self.config.model_output)
            # tensorboard
            self.add_summary(sess)
            for epoch in range(self.config.nepochs):
                self.logger.info("Epoch {:} out of {:}".format(epoch + 1, self.config.nepochs))

                acc, f1 = self.run_epoch(sess, train, dev, tags, epoch)

                # decay learning rate
                self.config.lr *= self.config.lr_decay

                # early stopping and saving best parameters
                if f1 >= best_score:
                    nepoch_no_imprv = 0
                    if not os.path.exists(self.config.model_output):
                        os.makedirs(self.config.model_output)
                    saver.save(sess, self.config.model_output)
                    best_score = f1
                    self.logger.info("- new best score!")

                else:
                    nepoch_no_imprv += 1
                    if nepoch_no_imprv >= self.config.nepoch_no_imprv:
                        self.logger.info("- early stopping {} epochs without improvement".format(
                                        nepoch_no_imprv))
                        break


    def evaluate(self, test, tags):
        saver = tf.train.Saver()
        with tf.Session() as sess:
            self.logger.info("Testing model over test set")
            saver.restore(sess, self.config.model_output)
            acc, f1 = self.run_evaluate(sess, test, tags)
            self.logger.info("- test acc {:04.2f} - f1 {:04.2f}".format(100*acc, 100*f1))

In [ ]:
def main(config):
    # load vocabs
    vocab_words = load_vocab(config.words_filename)
    vocab_tags  = load_vocab(config.tags_filename)
    vocab_chars = load_vocab(config.chars_filename)

    # get processing functions
    processing_word = get_processing_word(vocab_words, vocab_chars,
                    lowercase=True, chars=config.chars)
    processing_tag  = get_processing_word(vocab_tags, 
                    lowercase=False)

    # get pre trained embeddings
    embeddings = get_trimmed_glove_vectors(config.trimmed_filename)

    # create dataset
    dev   = CoNLLDataset(config.dev_filename, processing_word,
                        processing_tag, config.max_iter)
    test  = CoNLLDataset(config.test_filename, processing_word,
                        processing_tag, config.max_iter)
    train = CoNLLDataset(config.train_filename, processing_word,
                        processing_tag, config.max_iter)

    # build model
    model = NERModel(config, embeddings, ntags=len(vocab_tags),
                                         nchars=len(vocab_chars))
    model.build()

    # train, evaluate
    model.train(train, dev, vocab_tags)
    model.evaluate(test, vocab_tags)

First, build vocab from the data and extract trimmed glove vectors according to the config in `config.py`.
```
python build_data.py
```

In [ ]:
# create instance of config
config = Config()

# load, train, evaluate and interact with model
main(config)

To train the model and run it in interactive mode on shell:
```
python main.py
```